## HEAD

#### HEAD01 - load libraries

In [1]:
##########==========##########==========##########==========##########==========
import tweepy
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

#### HEAD02 - load data files

In [2]:
## read in roster of handles
handle_roster = pd.read_excel("A_Input/twitter_handles.xlsx")

## read in twitter credentials; initialize api connection+
twitter_credentials = pd.read_csv('../api_keys/twitter.csv').set_index('item')
twitter_credentials = tweepy.OAuth1UserHandler(
    consumer_key = twitter_credentials.loc['API Key', 'string'],
    consumer_secret = twitter_credentials.loc['API Key Secret', 'string'],
    access_token = twitter_credentials.loc['Access Token', 'string'],
   access_token_secret = twitter_credentials.loc['Access Token Secret', 'string']
    )
api = tweepy.API(twitter_credentials)

/Users/s8/opt/anaconda3/envs/py310/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## HAND

#### HAND01 - extract handles from roster URL

## PULL

#### PULL01 - query API for each roster handle's user_timeline data

In [27]:
x = api.user_timeline(screen_name = 'RepAdams', count = 200,
                      tweet_mode = 'extended',
                      exclude_replies = True, include_rts = False)

In [36]:
x[5].id

1549866361559580674

In [40]:
x[5].full_text

'Glad to see many of my Republican friends in the Senate supporting #EqualMarriage.\n\nI sincerely hope in the wake of Dobbs we can finally put the freedom to marry on the President’s desk and guarantee #LGBTQ marriage in every state.'

In [41]:
len(x)

94

In [6]:
handle_roster.head(1)

,#,group,party,member,state,handle
0,1,USA House,Democratic,"Adams, Alma",North Carolina,https://twitter.com/RepAdams
